# 关于解决Python乱码问题的终极解决方案

> 这里打算讨论各种常用方法

简单来说，只要记住，在Python2里字符串只有两大阵营：

## `unicode`和`bytes`

如果`type(字符串)`显示结果是`str`，那么就是指的`bytes`二进制码。
而其它各种我们所说的`utf-8`，`gb2312`等等也都是Unicode的不同实现方式。
这里不要去考虑那么复杂，只要先记住这两大阵营就行。

## `encoding`和`decoding`

绝对要记住的：
从`unicode`转换到`bytes`，这个叫`encoding`，编码。
从`bytes`转换到`unicode`，这个叫`decoding`，解码。

来回记住这个问题，才能进入下一步！

然后来看个案例。

In [74]:
'你好'

'\xe4\xbd\xa0\xe5\xa5\xbd'

In [75]:
u'你好'

u'\u4f60\u597d'

In [76]:
len('你好')

6

In [77]:
len(u'你好')

2

In [78]:
type('你好')

str

In [79]:
type(b'你好')

str

In [80]:
type(u'你好')

unicode

> 通过上面两种格式的对比我们看到，str和unicode的各种区别。

那么，既然变量里面会出现两种不同的格式，如果我们把两种格式的字符串连在一起操作会发生什么呢？
如下：

In [81]:
uni = u'你好'
type(uni)

unicode

In [82]:
byt = b'你好'
type(byt)

str

In [83]:
uni + byt

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe4 in position 0: ordinal not in range(128)

> ### 看！著名的`UnicodeDecodeError: 'ascii' codec can't decode byte`编码错误就这样出现了！

以上是我们用`显性`字符串来比较两种格式字符串的区别。

但是，我们经常性处理python编码问题，都不是在这种`显性`的字符串上出现的，不是从网上爬取的就是从本地文件读取的，意思就是文件内容庞大，编码格式很难猜到是什么。
所以这里我们将问题再拆分为两部分讨论：本地文件和网络资源。

## 本地文件编码测试
首先在本地建立一个有中文的以`utf-8`格式保存的文本文件（实际上无论.txt还是.md等都无所谓，内容是一样的）。
内容只有'你好'。

### 然后我们来读取一下：

In [85]:
with open('test.txt', 'r') as f:
    ss = f.read()

In [86]:
type(ss)

str

> 上面看到，从文件读取出来的，就是bytes二进制格式。
那么如果要把bytes转化为unicode，就要解码，也就是decoding.

In [87]:
ss.decode('utf-8')

u'\u4f60\u597d'

### 这种时候实际上是最迷糊也最容易造成之后错误的，就是分不清该编码还是该解码。

> 所以上面提到，必须要记住这两个区别。
那么如果现在我搞反了怎么办？就会再次出现下面错误：

In [88]:
ss.encode('utf-8')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe4 in position 0: ordinal not in range(128)

### 话说回来，我们该怎么统一他们呢？
> 为了避免两种格式的字符串在一起乱搞，统一他们是必须的。但是以哪一种为统一的呢，unicode还是bytes?

#### 下面我们来看看做常用的环境下字符串都是什么格式

In [89]:
# 文中声明的变量
type('你好')

str

In [90]:
# 读取本地文件
ff = open('test.txt', 'r')
type(ff.read())

str

In [91]:
# 获取网路资源
import requests
r = requests.get('http://pycoders-weekly-chinese.readthedocs.io/en/latest/issue5/unipain.html')
print type(r.text)
print type(r.content)

<type 'unicode'>
<type 'str'>


> 这样就明白了：除了r.text返回的内容外，其它几乎都是使用str格式，也就是bytes二进制码。所以我们只要转化requests相关的内容就行！

实际上，requests返回的response中, 除了用`response.text`获取内容，我们还可以用`response.content`获取同样的内容，只不过是bytes格式。

那就正和我们意，不用再去转化每一个地方的字符串，而只要盯紧这一个地方就足够了。

### 为什么我们不能把所有字符串变量统一为unicode呢？
提前说明，变成unicode的过程，叫`decoding`。不要记错。

因为像`response.text`经常把`ISO8859`等猜不到也检测不到编码(机率很低)的字符串扔过来，如果遇到的话，是很麻烦的。

`decoding`有两种方法：
```
unicode(b'你好‘）
b'你好'.decode('utf-8')
```

这里因为不知道来源的编码，所以必须用`unicode()`来解码，而不能用`.decode('utf-8')`，因为显然你不能乱写解码名称，如果来源果真是（很大几率是）`ISO8859`等方式，那么错误的解码肯定会产生乱码，或者直接程序报错。切记！

所以这里只能用`unicode()`解码。如下例：

In [124]:
type(unicode(r.text))

unicode

这里好像没问题，变成了unicode。但是！但是一旦我们需要输出字符串，那么就必须要str格式的才能输出。

这时候我们怎么办呢？只能用`bytes()`再把它转成str格式，同理，因为不知道来源的编码格式，不能用`.encode('utf-8')`这个方法来转换。

In [123]:
# 来看看
type(bytes( unicode(r.text) ))


UnicodeEncodeError: 'ascii' codec can't encode characters in position 276-281: ordinal not in range(128)

## 本阶段总结：一定记住，全文都统一用`str`格式字符串
### 只要盯紧requests等相关的网络操作就好了，只要控制好外来源的字符串，统一为`str`，其它一切都好说！